<a href="https://colab.research.google.com/github/TerriDonut/Yandex-educational-projects/blob/main/machine_learning_in_business.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
**Выбор локации для скважины**
---
---
**Описание проекта**

В добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации:
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
Строят модель для предсказания объёма запасов в новых скважинах;
Выбирают скважины с самыми высокими оценками значений;
Определяют регион с максимальной суммарной прибылью отобранных скважин.
Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны.

---
**Цель проекта**

Построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализировав возможную прибыль и риски техникой Bootstrap.

**Задачи проекта**

- Построить модель линейной регрессии (остальные — недостаточно предсказуемые).
- Выбрать 200 лучших точек для разрабоки, из 500 исследованных при разведке региона.
- При нынешних ценах на один баррель сырья приносящий 450 рублей дохода, подобрать оптимальрый бюджет на разработку скважин в регионе — 10 млрд рублей.
- Оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Выбрав регион с наибольшей средней прибылью.
---

**Исходные данные**

Данные геологоразведки трёх регионов находятся в файлах:

*datasets/geo_data_0.csv*

*datasets/geo_data_1.csv*

*datasets/geo_data_2.csv*

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек;

product — объём запасов в скважине (тыс. баррелей).
---

**Содержание проекта**

- ***Шаг 1. Загрузка и подготовка данных:***
** [Загрузка библиотек.](#1-bullet)
** [Изученеие файлов с данными, получение общей информации.](#2-bullet)
** [Поиск и устранение дублей.](#3-bullet)
** [Построение пайплайна предобработки.](#4-bullet)
- ***Шаг 2. Обучение и проверка модели для каждого региона:***
** [Сохранение результатов обучения](#5-bullet)

- ***Шаг 3. Подготовка к расчёту прибыли:***
** [Сравнение средних значений для безубыточной добычи](#6-bullet)

- ***Шаг 4. Расчёт прибыли по выбранным скважинам и предсказаниям модели***

- ***Шаг 5. Подсчёт рисков и прибыли для каждого региона***
** [Bootstrap](#7-bullet)
** [Финальные метрики и вывод](#8-bullet)


##Шаг 1. Загрузка и подготовка данных:

### Загрузка библиотек
<a id='1-bullet'></a>

In [ ]:
#Импорты базовых библиотек
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

from warnings import simplefilter

In [ ]:
simplefilter(action='ignore', category=FutureWarning)

### Изученеие файлов с данными, получение общей информации
<a id='2-bullet'></a>

In [ ]:
try:
    df_1 = pd.read_csv('/content/geo_data_0.csv')#, sep=',')
    df_2 = pd.read_csv('/content/geo_data_1.csv')#, sep=',')
    df_3 = pd.read_csv('/content/geo_data_2.csv')#, sep=',')
except:
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')#, sep=',')
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')#, sep=',')
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')#, sep=',')

#Просмотр
display(df_1.info())
display(df_1.head(5))

display(df_2.info())
display(df_2.head(5))

display(df_3.info())
(df_3.head(5))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


### Поиск и устранение дублей
<a id='3-bullet'></a>

In [ ]:
#Функция по поиску дублей
def search_duplicates(df):
    print('Количество дубликатов в столбцах:')
    display(df.apply(lambda col: col.duplicated().sum()))

    print('Количество дубликатов в строках:')
    display(df.duplicated().sum())


display(search_duplicates(df_1))
display(search_duplicates(df_2))
display(search_duplicates(df_3))

Количество дубликатов в столбцах:


id         10
f0          0
f1          0
f2          0
product     0
dtype: int64

Количество дубликатов в строках:


0

None

...

Количество дубликатов в столбцах:


id         4
f0         0
f1         0
f2         0
product    0
dtype: int64

Количество дубликатов в строках:


0

None

<b>Комментарий к предобработке</b>

- Дубли в столбцах 'id' можно удалить, ведь должно быть уникальное значение
- Повторы в объёме запасов вполне объяснимы округлением  

In [ ]:
def prepr(df):
    df = df.drop_duplicates(subset='id')

    df = df.drop('id', axis=1)

    df = df.dropna()

    return df


df_1 = prepr(df_1)
df_2 = prepr(df_2)
df_3 = prepr(df_3)


### Построение пайплайна предобработки
<a id='4-bullet'></a>

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

#Список признаков для масштабирования
num_columns = ['f0', 'f1', 'f2']

#Общий пайплайн предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns)
])

##Шаг 2. Обучение и проверка модели для каждого региона

In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

def pred(data):
    X = data.drop(['product'], axis=1)
    y = data['product']

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

    pipeline.fit(X_train, y_train)

    predictions = pipeline.predict(X_valid)

    mse = mean_squared_error(y_valid, predictions)
    rmse = np.sqrt(mse)

    return predictions, y_valid, rmse

df_list = [df_1, df_2, df_3]
results = {}

#Цикл для обучения моделей и сохранения результатов в словарь
for i, df in enumerate(df_list):
    predictions, y_valid, rmse = pred(df)
    results[f"Регион {i}"] = {'Средний предсказанный запас сырья': predictions.mean().round(2), 'RMSE': rmse.round(2)}

#Вывод среднего запаса предсказанного сырья и RMSE модели для каждого региона
results

{'Регион 0': {'Средний предсказанный запас сырья': 92.61, 'RMSE': 37.69},
 'Регион 1': {'Средний предсказанный запас сырья': 68.58, 'RMSE': 0.89},
 'Регион 2': {'Средний предсказанный запас сырья': 94.93, 'RMSE': 40.08}}

<b>Вывод к обучению</b>

- Лучше всего модель обучилась на данных первого региона, разброс минимален, но средний запас сырья ниже чем у двух других

###Сохранение результатов обучения
<a id='5-bullet'></a>

In [ ]:
#Сохранение результатов обучения для расчётов
predictions_region1, y_valid_region1, rmse_region1 = pred(df_1)
predictions_region2, y_valid_region2, rmse_region2 = pred(df_2)
predictions_region3, y_valid_region3, rmse_region3 = pred(df_3)


##Шаг 3. Подготовка к расчёту прибыли

In [ ]:
#Бюджет на разработку 200 скважин
budget_per_well = 10e9
# Доход с одной тысячи баррелей нефти
barrel_revenue = 450 * 1000
#Количество скважин для разработки
wells_to_drill = 200

#Расчёт безубыточного объёма
breakeven_volume = budget_per_well / (barrel_revenue * wells_to_drill)


###Сравнение средних значений для безубыточной добычи
<a id='6-bullet'></a>

In [ ]:
print('Количество тыс. баррелей в скважине для безубытончной разраотки:', breakeven_volume)

for i, df in enumerate(df_list):
    mean_value = df['product'].describe()
    print(f'Количество баррелей в регионе {i+1}: {mean_value}')

Количество тыс. баррелей в скважине для безубытончной разраотки: 111.11111111111111
Количество баррелей в регионе 1: count    99990.000000
mean        92.499684
std         44.288304
min          0.000000
25%         56.497069
50%         91.847928
75%        128.563699
max        185.364347
Name: product, dtype: float64
Количество баррелей в регионе 2: count    99996.000000
mean        68.823916
std         45.944663
min          0.000000
25%         26.953261
50%         57.085625
75%        107.813044
max        137.945408
Name: product, dtype: float64
Количество баррелей в регионе 3: count    99996.000000
mean        94.998342
std         44.749573
min          0.000000
25%         59.450028
50%         94.925026
75%        130.586815
max        190.029838
Name: product, dtype: float64


<b>Вывод к расчёту прибыли</b>

- Среднее количество тыс. баррелей не дотягивает до безубыточных границ, но это вполне компенсируется квартилями 75%

##Шаг 4. Расчёт прибыли по выбранным скважинам и предсказаниям модели

In [ ]:
#Функция для расчёта дохода в регионах
def calculate_profit(predictions, y_valid):
    sorted_predictions = np.sort(predictions)[::-1]
    selected_wells_indices = np.argsort(predictions)[::-1][:wells_to_drill]
    total_oil_volume = y_valid.iloc[selected_wells_indices].sum()
    total_revenue = total_oil_volume * barrel_revenue - budget_per_well

    return total_revenue

#Сохранение результатов
profit_region1 = calculate_profit(predictions_region1, y_valid_region1)
profit_region2 = calculate_profit(predictions_region2, y_valid_region2)
profit_region3 = calculate_profit(predictions_region3, y_valid_region3)

#Вывод
print(profit_region1, profit_region2, profit_region3)

3468529787.422388 2415086696.681511 2396613000.939848


<b>Вывод к расчёту регионального дохода</b>

Все регионы потенциально принесут выручку

##Шаг 5. Подсчёт рисков и прибыли для каждого региона

###Bootstrap
<a id='7-bullet'></a>

In [ ]:
#Функция с Bootstrap с 1000 выборок, для распределения прибыли
def bootstrap_profit(predictions, y_valid):
    state = np.random.RandomState(42)
    profits = []
    for _ in range(1000):
        random_indices = state.choice(len(predictions), size=500, replace=True)
        bootstrap_predictions = predictions[random_indices]
        bootstrap_y_valid = y_valid.iloc[random_indices]

        profit = calculate_profit(bootstrap_predictions, bootstrap_y_valid)
        profits.append(profit)

    return profits

#Cохранение результатов
bootstrap1 = bootstrap_profit(predictions_region1, y_valid_region1)
bootstrap2 = bootstrap_profit(predictions_region2, y_valid_region2)
bootstrap3 = bootstrap_profit(predictions_region3, y_valid_region3)


###Финальные метрики и вывод
<a id='8-bullet'></a>

In [ ]:
#Функция для расчёта метрик
def cal_metrics(bootstrap, reg_name):
    bootstrap = pd.Series(bootstrap)

    mean_prof = np.mean(bootstrap)

    lower = np.percentile(bootstrap, 2.5)
    upper = np.percentile(bootstrap, 97.5)

    losses = np.sum(np.array(bootstrap) < 0) / len(bootstrap)

    print(reg_name)
    print(f'Средняя прибыль: {mean_prof}')
    print(f'95%-й доверительный интервал: [{lower}, {upper}]')
    print(f'Риск убытков: {losses}')


cal_metrics(bootstrap1, 'Регион 1')
cal_metrics(bootstrap2, 'Регион 2')
cal_metrics(bootstrap3, 'Регион 3')


Регион 1
Средняя прибыль: 403572063.9550541
95%-й доверительный интервал: [-137690003.1514729, 902848310.7781234]
Риск убытков: 0.059
Регион 2
Средняя прибыль: 430208352.49343604
95%-й доверительный интервал: [43052519.63423143, 847313467.2140586]
Риск убытков: 0.016
Регион 3
Средняя прибыль: 384214428.6827778
95%-й доверительный интервал: [-127589876.98701142, 910134439.5370146]
Риск убытков: 0.08


<b>Финальный вывод</b>

- Разработкой пердлагается заняться в 2 регионе, потенциально он может принести наибольшую прибыль, и значения доверительного интерва находятся хоть и не в столь стабильных, но достаточно высоких (В сравнении с другими регионами) диапазонах значений